In [9]:
pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.24.2)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install dictances

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [11]:
from __future__ import division
import json
import numpy as np
from decimal import Decimal
from numpy import expand_dims, array, exp, max
import scipy
from scipy.special import rel_entr
import csv
import statistics
from numpy import expand_dims, array, exp, max
import math
from scipy.spatial import distance
from scipy.stats import wasserstein_distance
import numpy as np
from numpy.linalg import norm
import sklearn
from tensorflow import keras
import string
from scipy.special import rel_entr
from tensorflow.keras.applications.resnet50 import ResNet50 as resnet50
from tensorflow.keras.applications.vgg16 import VGG16 as vgg16
from tensorflow.keras.applications.densenet import DenseNet169 as densenet
from tensorflow.keras.applications.inception_v3 import InceptionV3 as inceptionv3
from tensorflow.keras.applications.efficientnet import EfficientNetB1 as efficient
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 as mobilenetv2
from tensorflow.keras.applications.mobilenet import MobileNet as mobilenet
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations
import csv
# import the timeit module
import timeit
import tracemalloc

In [12]:
from numpy.linalg import norm
from dictances import bhattacharyya, bhattacharyya_coefficient

In [13]:
# import the timeit module
import timeit
import math
import tracemalloc

In [14]:
def hellinger_fast(p, q):
    """Hellinger distance between two discrete distributions.
       In pure Python.
       Fastest version.
    """
    return sum([ (math.sqrt(p_i) - math.sqrt(q_i))**2 for p_i, q_i in zip(p,q) ])

In [15]:
# to suppress division errors
np.seterr(divide='ignore', invalid='ignore')
new_final_source_array = []

In [48]:
# Opening final JSON files
contrast_source_data = open('../paper3/files/glcm/source_domain_data_new_energy_glcm.json')

#for target data
contrast_target_data = open('files/Caltech256/DenseNet169/glcm/caltech_domain_data_new_energy_glcm.json')

In [49]:
# returns JSON object
contrast_source_data_d = json.load(contrast_source_data)

contrast_target_data_d = json.load(contrast_target_data)

In [50]:
# kdl dictionary
kdl_dictionary = {}

In [51]:
def KL(p, q):
    a = []
    b = []
    for k in p:
        a.append(Decimal(k))

    for l in q:
        b.append(Decimal(l))

    a = np.asarray(a, dtype=np.float64)
    b = np.asarray(b, dtype=np.float64)

    return np.sum(np.where(((a != 0) & (b != 0)), a * np.log(a / b), 0))

In [52]:
def softMax(temperature, features_list):
    feat_list = [x / temperature for x in features_list]
    feature_list = exp(feat_list - max(feat_list))
    return feature_list / feature_list.sum(axis=0)

In [53]:
# get the data for the contrast target image
contrast_target_data_array = []
for p in range(0, len(contrast_target_data_d['target_images'])):
    for i, j in contrast_target_data_d['target_images'][p].items():
        contrast_target_data_array.append((i,np.array(j)))
contrast_target_data.close()

In [54]:
print(len(contrast_target_data_array))

7710


In [55]:
###contrast comparison code
contrast_All_Images = []
contrast_average_dkl = []

#check the timer_start
start_hellinger = timeit.default_timer()
#check memory consumption
# starting the monitoring
tracemalloc.start()

for x in contrast_target_data_array[:1]:
#for x in target_data_array:
    t_images_btween_0_05 = []
    t_images_btween_05_1 = []
    t_images_btween_1_2 = []
    t_images_grt_2_5 = []
    t_images_grt_5_10 = []
    t_images_grt_10 = []
    
    image_name_class =  x[0].split('_/') #name[0]; class[1]
    compare_min_max = []
    
    
            
    # get the data for the source images
    for m in range(0, len(contrast_source_data_d['source_images'])):
        #print("source",source_data_d['source_images'][m][0])
        for k, v in contrast_source_data_d['source_images'][m].items():
            # compare the value of v with that of the target image
            # if 'caterpillar' in k.lower():
            # remove the zeros from the list
            new_v = np.array(v).flatten()
            #get the top items in x
            items = np.sort(x[1])
            item_size = items[-len(new_v):]
            new_x = item_size.flatten()
            
            new_d = softMax(0.5, new_x)     
            new_y = softMax(0.5, new_v)
            
            kdl = rel_entr( new_x, new_v, out=None)
            new_bhattacharyya = bhattacharyya(dict(enumerate(new_x.flatten(), 1)),dict(enumerate(new_v.flatten(), 1)) )
            new_wasserstein = wasserstein_distance(new_x,new_v)
            new_jensen = distance.jensenshannon(new_x,new_v)
            new_hellinger = hellinger_fast(new_x,new_v)
            
            compare_min_max.append((image_name_class[0],image_name_class[1],k, max(new_bhattacharyya)))
            
contrast_All_Images.append(compare_min_max)
    
contrast_source_data.close()

stop = timeit.default_timer()
execution_time = stop - start_hellinger
print("KDL -selected points execution time:-", execution_time)

# displaying the memory
print("used memory",tracemalloc.get_traced_memory()[1]-tracemalloc.get_traced_memory()[0])
 
# stopping the library
tracemalloc.stop()

model_name = densenet
input_t = keras.Input(shape=(224, 224, 3))
model = model_name(include_top=False,
                   weights="imagenet",
                   input_tensor=input_t)


# get the layer index
def getLayerIndex(model_i, layer_name):
    for pos, layer_g in enumerate(model_i.layers):
        if layer_g.name == layer_name:
            return pos


# get the convolved layers into an array for looping
convolved_layers = []
convolved_layer_names = []
# create a list to place them
layer_cosine_list = []
layer_final_cos_list = []

for layer in model.layers[0:-2]:
    if 'conv' not in layer.name:
        continue

    index = getLayerIndex(model, layer.name)
    #print(layer.name)
    # append the convolved layer
    convolved_layers.append(index)
    convolved_layer_names.append(layer)

jdk = {}
for lyr in convolved_layer_names:
    ary = np.array(lyr.get_weights(), dtype=object)
    layer_index = getLayerIndex(model, lyr.name)
    if len(ary) != 0:
        # filters, biases = layer.get_weights()
        # print(filters.shape)
        ary = np.array(lyr.get_weights(), dtype=object)

        zipper_dict = {}
        dict_array = []

        # check for the arrays
        for x in ary:
            # print(len(x))
            # find if the array is 1 dim
            if x.ndim > 1:
                for y in x:
                    for z in y:
                        u = 0
                        for za in z:
                            # get all the positive elements in this and push them into an array
                            positive_elements = []
                            for pos_item in za:
                                if pos_item > 0:
                                    positive_elements.append(pos_item)
                            u = u + 1
                            zipper_dict.update({u: positive_elements})

            elif x.ndim == 1:
                # get the items/arrays with more than the 1 dimension
                print("")
        # print("lyr:",lyr.name)
        # get two items per tuple
        for i in range(1, len(zipper_dict), 2):
            if len(zipper_dict[i]) > 0:
                if len(zipper_dict[i]) == len(zipper_dict[i + 1]):

                    new_i = np.array(zipper_dict[i]).reshape(1, -1)
                    new_i_plus_1 = np.array(zipper_dict[i + 1]).reshape(1, -1)
                    cos_item = np.ravel(cosine_similarity(new_i, new_i_plus_1))
                    layer_cosine_list.append(cos_item)

                elif len(zipper_dict[i]) > len(zipper_dict[i + 1]):

                    diff_len = len(np.array(zipper_dict[i])) - len(np.array(zipper_dict[i + 1]))
                    old_i = np.pad(zipper_dict[i + 1], (0, diff_len), 'constant')

                    new_i = old_i.reshape(1, -1)
                    new_i_plus_1 = np.array(zipper_dict[i]).reshape(1, -1)

                    cos_item = np.ravel(cosine_similarity(new_i, new_i_plus_1))
                    layer_cosine_list.append(cos_item)

                elif len(zipper_dict[i]) < len(zipper_dict[i + 1]):
                    # print("######less than i plus 1#########")

                    diff_len = len(zipper_dict[i + 1]) - len(zipper_dict[i])
                    old_i = np.pad(zipper_dict[i], (0, diff_len), 'constant')
                    new_i = old_i.reshape(1, -1)
                    new_i_plus_1 = np.array(zipper_dict[i + 1]).reshape(1, -1)

                    cos_item = np.ravel(cosine_similarity(new_i, new_i_plus_1))
                    layer_cosine_list.append(cos_item)


        # calculate softmax function
        def softmax(s_et):
            e_set = np.exp(s_et - np.max(s_et))
            return e_set / e_set.sum(axis=0)


        for a in layer_cosine_list:
            layer_final_cos_list.append(a)
        # print(layer_final_cos_list)
        layer_softmax_values = softmax(np.ravel(np.array(layer_cosine_list, dtype=np.float32).reshape(1, -1)))
        jdk.update({layer_index: layer_softmax_values})

# jdk gives a dictionary made up of probability distributions
# print(jdk)
# print("the other one is")
b = [x for y, x in enumerate(jdk) if y != i]
all_items = list(combinations(b, 2))

# put the layers in a dictionary and rank them
other = []
#check timer here
# start the timer
start_bhattacharyya = timeit.default_timer()
#check memory consumption
# starting the monitoring
tracemalloc.start()
        
for a in all_items:
    if len(jdk[a[0]]) > len(jdk[a[1]]):
        
        dif_1 = len(np.array(jdk[a[0]])) - len(np.array(jdk[a[1]]))
        new_jdk = np.pad(np.array(jdk[a[1]]), (0, dif_1), 'constant').reshape(1, -1)
        new_i_jdk = np.array(jdk[a[0]]).reshape(1, -1)
        #print("########################other###############") 
        new_bhattacharyya = bhattacharyya(new_jdk[0],new_i_jdk[0])
        #print(new_bhattacharyya)
        #print("##########################end other#########")
        
        #hellinger_fast(new_i_jdk, new_jdk)
        ##kl_pq = rel_entr(np.ravel(np.ravel(new_i_jdk), new_jdk))
       # print("layer:", a[0], " and layer: ", a[1], ':KL(P ||2 Q): %.3f nats1' % sum(kl_pq))
        ##########m_no = [a[0], a[1], hellinger_fast]
        ##########other.append(m_no)

    elif len(jdk[a[0]]) < len(jdk[a[1]]):
        #check timer here
        dif_1 = len(np.array(jdk[a[1]])) - len(np.array(jdk[a[0]]))
        new_jdk = np.pad(np.array(jdk[a[0]]), (0, dif_1), 'constant').reshape(1, -1)
        new_i_jdk = np.array(jdk[a[1]]).reshape(1, -1)
        new_jdk_dictionary = { i : new_jdk[0][i] for i in range(0, len(new_jdk[0]) ) }
        new_i_jdk_dictionary = { i : new_i_jdk[0][i] for i in range(0, len(new_i_jdk[0]) ) }
        
        #print("########################other###############")        
        new_bhattacharyya = bhattacharyya(new_jdk_dictionary,new_i_jdk_dictionary)
        print(new_bhattacharyya)
        #print("##########################end other#########")       
        

        #kl_pq = rel_entr(np.ravel(new_jdk), np.ravel(new_i_jdk))
        #print("layer:", a[0], " and layer: ", a[1], ':KL(P ||2 Q): %.3f nats' % sum(kl_pq))
        #m_no = [a[0], a[1], sum(kl_pq)]
        ###########hellinger_fast(new_i_jdk, new_jdk)
        ##kl_pq = rel_entr(np.ravel(np.ravel(new_i_jdk), new_jdk))
       # print("layer:", a[0], " and layer: ", a[1], ':KL(P ||2 Q): %.3f nats1' % sum(kl_pq))
        ###########m_no = [a[0], a[1], hellinger_fast]
        ##############other.append(m_no)
stop_bhattacharyya = timeit.default_timer()
execution_time = stop_bhattacharyya - start_bhattacharyya
print("bhattacharyya execution time:-", execution_time)

# displaying the memory
print("used memory",tracemalloc.get_traced_memory()[1]-tracemalloc.get_traced_memory()[0])
 
# stopping the library
tracemalloc.stop()

# with open('mobilenet_kullback_positive.csv', 'w', encoding='UTF8', newline='') as f:
#     writer = csv.writer(f)  # write the header
#     writer.writerow(["Layer1", "Layer2", "DKL"])
#     for l_item in other:
#         writer.writerow(l_item)
    # All the program statements




KDL -selected points execution time:- 3.956636567017995
used memory 12299




















































































































































































































































































































































































































































































































































































































































































1.5968668079712496
1.5968668079712496
2.132182381366069
2.132182381366069
2.33281534900962
2.33281534900962
2.5194319858383625
2.5194319858383625
2.6495497700095494
2.6495497700095494
2.753400322174198
2.753400322174198
2.859020998753005
2.859020998753005
2.93072346

4.024970476266062
4.032429040704399
4.032429040704399
4.052041135761671
4.052041135761671
4.058898989941674
4.058898989941674
4.079042629400523
4.079042629400523
4.08564631683823
4.08564631683823
4.106635414505547
4.106635414505547
4.112730921548969
4.112730921548969
4.134050830595409
4.134050830595409
4.140200055571368
4.140200055571368
4.162044173581533
4.162044173581533
4.167746306551045
4.167746306551045
4.189560274271338
4.189560274271338
4.194842405112473
4.194842405112473
4.217035558307909
4.217035558307909
4.222174009995177
4.222174009995177
4.244112389168642
4.244112389168642
4.248909656825907
4.248909656825907
4.271244379307025
4.271244379307025
4.275880451696531
4.275880451696531
4.298288248712185
4.298288248712185
4.302579102648554
4.324393799839056
4.324393799839056
4.32830750202005
4.32830750202005
4.349904215293163
4.349904215293163
4.353873858606688
4.353873858606688
4.3753437715690815
4.3753437715690815
4.3789563940612535
4.3789563940612535
4.400547641040001
4.40054764

4.776374386516923
4.776374386516923
4.785529975563075
4.785529975563075
4.7869907962833125
4.7869907962833125
4.796362948729675
4.796362948729675
4.797839714221067
4.797839714221067
4.807471547127079
4.807471547127079
4.808967102232949
4.808967102232949
4.818901955911133
4.818901955911133
4.820406856644295
4.820406856644295
4.830339301602074
4.830339301602074
4.831714166259336
4.831714166259336
4.841812220368164
4.841812220368164
4.843104914938355
4.843104914938355
4.853374036803405
4.853374036803405
4.854617858953284
4.854617858953284
4.864957326216742
4.864957326216742
4.866221261449862
4.866221261449862
4.876611443981741
4.876611443981741
4.877764387012644
4.877764387012644
4.888221593810928
4.888221593810928
4.889353836524098
4.889353836524098
4.9001040818623265
4.9001040818623265
4.901234121009959
4.901234121009959
4.911877120664304
4.911877120664304
4.9129089806287505
4.9129089806287505
4.923725610217451
4.923725610217451
4.924721576756221
4.924721576756221
4.93553578130986
4.935

3.5254137043458136
3.5254137043458136
3.526062865330619
0.5353155903819931
0.5353155903819931
0.7359485587533533
0.7359485587533533
0.9225651912606365
0.9225651912606365
1.052682968615104
1.052682968615104
1.1565335201894642
1.1565335201894642
1.2621542039807003
1.2621542039807003
1.3338566782008006
1.3338566782008006
1.4238211218330081
1.4238211218330081
1.477048835199624
1.477048835199624
1.5549722781761246
1.5549722781761246
1.5959921799155485
1.6609637712144685
1.6609637712144685
1.6921391693753962
1.6921391693753962
1.723530271403131
1.723530271403131
1.7594624381873192
1.7594624381873192
1.7874247389641609
1.7874247389641609
1.825901148162674
1.825901148162674
1.8499108964701965
1.8499108964701965
1.8893878722117738
1.8893878722117738
1.9107804030359004
1.9107804030359004
1.9508971305830722
1.9508971305830722
1.9700254607688088
1.9700254607688088
2.010022486564932
2.010022486564932
2.0273799427655437
2.0273799427655437
2.0663933163481283
2.0663933163481283
2.0816082153927122
2.08

2.139062051461778
2.143698137324387
2.143698137324387
2.1661059343931752
2.1661059343931752
2.170396788837823
2.192211506325179
2.192211506325179
2.196125177457232
2.196125177457232
2.217721891043941
2.217721891043941
2.2216915510687154
2.2216915510687154
2.24316145787756
2.24316145787756
2.246774068073207
2.246774068073207
2.2683653451725467
2.2683653451725467
2.271877788967389
2.271877788967389
2.2930952038458194
2.2930952038458194
2.2963455279031355
2.2963455279031355
2.3176105546568846
2.3176105546568846
2.3207753589161313
2.3207753589161313
2.3418969388282926
2.3418969388282926
2.3448486129061172
2.3448486129061172
2.3657565167385193
2.3657565167385193
2.3685907367808436
2.3685907367808436
2.389165926826329
2.389165926826329
2.391879379722589
2.391879379722589
2.4122211987479476
2.4122211987479476
2.4147494239707843
2.4147494239707843
2.4350417072017256
2.4350417072017256
2.437461269827038
2.437461269827038
2.4573741774801263
2.4573741774801263
2.4596094017230636
2.459609401723063

...

0.3315906205689859
0.3315906205689859
0.3422336178055696
0.3422336178055696
0.34326547122874185
0.34326547122874185
0.35408207911717027
0.35408207911717027
0.35507805331297665
0.35507805331297665
0.3658922672247034
0.3658922672247034
0.366843882913558
0.366843882913558
0.37770270567070974
0.37770270567070974
0.3786128107013158
0.3786128107013158
0.38948647044565404
0.38948647044565404
0.3903470384631737
0.3903470384631737
0.4013376418457454
0.4013376418457454
0.40223880205744567
0.40223880205744567
0.4131465227603457
0.4131465227603457
0.4139819489588549
0.4139819489588549
0.42491816720106185
0.42491816720106185
0.4257324575623027
0.4257324575623027
0.43666650632033227
0.43666650632033227
0.43745087762074447
0.43745087762074447
0.44840816376157283
0.44840816376157283
0.4491773806058099
0.4491773806058099
0.4601269886048167
0.4601269886048167
0.46085584606323005
0.46085584606323005
0.4718400111751179
0.4718400111751179
0.47257334752124386
0.47257334752124386
0.48349938995655184
0.483499

0.17239867155551145
0.17239867155551145
0.17404220069928825
0.17404220069928825
0.18302102328795636
0.18302102328795636
0.18458275287543155
0.18458275287543155
0.1937383447963705
0.1937383447963705
0.19519915773198832
0.19519915773198832
0.204571282982912
0.204571282982912
0.2060480640596755
0.2060480640596755
0.21567987791737328
0.21567987791737328
0.21717545479457434
0.21717545479457434
0.22711031835792372
0.22711031835792372
0.228615224170695
0.228615224170695
0.23854765689332327
0.23854765689332327
0.23992253743486566
0.23992253743486566
0.2500205943371004
0.2500205943371004
0.25131325767429114
0.25131325767429114
0.2615823793263169
0.2615823793263169
0.2628262042770609
0.2628262042770609
0.2731656789080671
0.2731656789080671
0.27442960314354115
0.27442960314354115
0.28481979689256043
0.28481979689256043
0.2859727250277684
0.2859727250277684
0.2964299307336935
0.2964299307336935
0.29756218595963047
0.29756218595963047
0.30831241484995286
0.30831241484995286
0.30944248389032836
0.30

0.47650365952189516
0.47650365952189516
0.4771491826046224
0.4771491826046224
0.4879909307514609
0.4879909307514609
0.48862356300487686
0.48862356300487686
0.49937216594975575
0.49937216594975575
0.500027697500555
0.500027697500555
0.5108561388343035
0.5108561388343035
0.5115052986566195
0.0021984297431270282
0.0021984297431270282
0.021715219116555017
0.021715219116555017
0.023784182867996426
0.023784182867996426
0.04305691818930393
0.04305691818930393
0.04504161252677987
0.04504161252677987
0.06375495583508317
0.06375495583508317
0.06568310379812026
0.06568310379812026
0.08425629955210484
0.08425629955210484
0.08613720378739885
0.08613720378739885
0.10459147704669977
0.10459147704669977
0.10638384282630892
0.10638384282630892
0.12450538898947863
0.12450538898947863
0.12619130815189092
0.14406787314578717
0.14406787314578717
0.15276596469326956
0.15276596469326956
0.15440949404605522
0.15440949404605522
0.16338831628725484
0.16338831628725484
0.16495004613984465
0.16495004613984465
0.1

0.36916716046057824
0.36916716046057824
0.370002586803076
0.370002586803076
0.3809388050734326
0.3809388050734326
0.38175309516046163
0.38175309516046163
0.39268714415400163
0.39268714415400163
0.39347151532042346
0.39347151532042346
0.40442880170588813
0.40442880170588813
0.4051980184593957
0.4051980184593957
0.41614762594807403
0.41614762594807403
0.416876483944439
0.416876483944439
0.42786064916653105
0.42786064916653105
0.4285939853753004
0.4285939853753004
0.43952002767519865
0.43952002767519865
0.4402634794032519
0.4402634794032519
0.45116621959814496
0.45116621959814496
0.45188404966137574
0.45188404966137574
0.4627932821553816
0.4627932821553816
0.463487332015151
0.463487332015151
0.4743051410046667
0.4743051410046667
0.4749506638551797
0.4749506638551797
0.4857924120189684
0.4857924120189684
0.4864250442370051
0.4864250442370051
0.4971736470087258
0.4971736470087258
0.4978291785904776
0.4978291785904776
0.5086576198980791
0.5086576198980791
0.5093067799118358
0.002068976805703

KeyboardInterrupt: 